# Overview
This notebook will walk you through how to set-up a GP from any given bank of spectra. 

The GPs come from the python package `george` and we "train" them using the package `emcee`. 

Once the GP is trained, we export it as a pickle object to then use with PTA data.

**If you are training from scratch, start here**

**If you have already trained and want to use the GP model, start at [Testing the GP](#Testing-the-GP)** after importing the required packages

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from multiprocessing import Pool, cpu_count
import os
import warnings
import sys, glob, time, pickle
from pprint import pp

os.environ["OMP_NUM_THREADS"] = "1"

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import h5py
import scipy.signal as ssig
import scipy.interpolate as interp

import scipy.linalg as sl
import scipy.special as ss
import scipy.constants as sc
import scipy.misc as scmisc
import scipy.integrate as si
import scipy.optimize as opt

import george
import george.kernels as kernels
import emcee, corner

import holodeck as holo
from holodeck import utils, plot
from holodeck.constants import YR
import holodeck.gps.gp_utils as gu

from pathlib import Path

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

In [ ]:
# np.seterr(all='raise')
# warnings.filterwarnings("error")

In [ ]:
SPECTRA_FILE_NAME = (
    "/Users/lzkelley/programs/nanograv/15yr_astro_libraries/"
    "2par_wider/sam_lib.hdf5"
)

NFREQS = 5
TEST_FRAC = 0.01
BURN_FRAC = 0.2
NWALKERS = 20
NSAMPLES = 1000
MPI = False
CENTER_MEASURE = "median"
KERNEL_TYPE = "ExpSquaredKernel"
KERNEL_OPTS = {}


# Load Spectra

    The first step is to load the bank of spectra. 
    Make sure to double check that dimensionality of the parameter space, and get the parameter limits.

In [ ]:
# Start with Spectra from Luke
spectra_file = Path(SPECTRA_FILE_NAME)
spectra = h5py.File(spectra_file, 'r')

In [ ]:
print(list(spectra.keys()))
param_names = spectra.attrs['param_names'].astype(str)
sample_params = spectra['sample_params']
print(param_names)
print(sample_params.shape)

gwb = spectra['gwb']
print(gwb.shape)
if np.count_nonzero(gwb) / gwb.size < 0.5:
    raise RuntimeError(f"Fraction of gwb > 0.0 :: {utils.frac_str(gwb[()] > 0.0)}")

print(spectra['fobs'].shape)

In [ ]:
for ii, key in enumerate(param_names):
    vals = sample_params[:, ii]
    print(f"{key:>20s} (min, max) = ({vals.min():+.2e}, {vals.max():+.2e})")

In [ ]:
xx = spectra['fobs'][()] * YR
gwb = spectra['gwb']
nsamp = gwb.shape[0]
fig, ax = plot.figax()
for ii in np.random.choice(nsamp, size=10, replace=False):
    yy = gwb[ii, :, :]
    plot.draw_med_conf(ax, xx, yy)
        
plt.show()

## Smooth the GWB Spectra

In [ ]:
gp_freqs, xobs, yerr, yobs, yobs_mean = gu.get_smoothed_gwb(
    spectra, NFREQS, TEST_FRAC, CENTER_MEASURE
)

In [ ]:
fig, ax = plot.figax(scale='lin')
nsamp = yobs.shape[0]
xx = gp_freqs
for ii in np.random.choice(nsamp, size=10, replace=False):
    ave = yobs[ii, :]
    cc, = ax.plot(xx, ave, alpha=0.75)
    cc = cc.get_color()
    for conf in [1.0, 2.0]:
        hi = ave + yerr[ii, :] / conf
        lo = ave - yerr[ii, :] / conf
        ax.fill_between(xx, lo, hi, alpha=0.2, color=cc)
    
plt.show()

## Construct GP kernels 

In [ ]:
import holodeck as holo
for ii, par in enumerate(param_names):
    print(ii, par, xobs[:, ii].min(), xobs[:, ii].max())

In [ ]:
# kernel = george.kernels.ExpSquaredKernel(1.0e2)
# gp = george.GP(kernel)
# fig, ax = plt.subplots()
# t = np.linspace(0.0, 10.0, 100)
# ns = 10
# zz = 0.0

# for yy in gp.sample(t, ns):
#     ax.plot(t, yy)
#     zz += np.count_nonzero(np.diff(np.sign(yy)))

# print(zz/ns)

# plt.show()

In [ ]:
gp_george, num_kpars = gu.create_gp_kernels(
    gp_freqs, param_names, xobs, yerr, yobs, KERNEL_TYPE, KERNEL_OPTS
)

## Train GP (fit kernel parameters to smoothed spectra at each frequency)

In [ ]:
gu.fit_kernel_params(
    gp_freqs, yobs_mean, gp_george, num_kpars, NWALKERS, NSAMPLES, BURN_FRAC, MPI,
    sample_kwargs=dict(skip_initial_state_check=True)
)


## Examine

In [ ]:
gp = gp_george[0]
print(gp.emcee_flatchain.shape)
fig, axes = plot.figax(nrows=3, scale='lin')
zz = gp.emcee_flatlnprob
smap = plot.smap(zz, cmap='Spectral')
colors = smap.to_rgba(zz)
for ii, ax in enumerate(axes):
    yy = gp.emcee_flatchain[:, ii]
    ax.scatter(1+np.arange(yy.size), yy, facecolor=colors, edgecolor='0.25', lw=0.1, alpha=0.25, s=5)
    
plt.show()

In [ ]:
gp = gp_george[0]
print(gp.emcee_flatchain.shape)
# import kalepy as kale
# kale.corner(gp.emcee_flatchain.T)
# gp.emcee_kernel_map
fig, axes = plot.figax(nrows=3, scale='lin')
zz = gp.emcee_flatlnprob
print(zz.shape, utils.stats(zz))
smap = plot.smap([0.0, 1.0], cmap='Spectral')
colors = smap.to_rgba(np.linspace(0.0, 1.0, zz.size))
for ii, ax in enumerate(axes):
    xx = gp.emcee_flatchain[:, ii]
    ax.scatter(xx, zz, c=colors, s=3, alpha=0.2)

cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
plt.colorbar(smap, cax=cax)    
plt.show()

## Prepare Predictions

In [ ]:
gps = gu.set_up_predictions(gp_george)
mean_pars = gu.mean_par_dict(gp_george)
pp(gp_george[0].par_dict)

In [ ]:
gpg = gp_george[0]
gpg.y.shape

In [ ]:
num_freqs = len(gps)
print(f"{num_freqs=}")
# mp = list(mean_pars.values())
# print(mp)

# pars = mp
nsamp = gp_george[0].y.size
idx = np.random.choice(nsamp)
print(f"{idx=}")

pars = spectra['sample_params'][idx, :]

print(pars)
xx = spectra['fobs'][:num_freqs]

hc, rho, rho_pred = gu.hc_from_gp(gp_george, gps, pars)
print(hc)

fig, ax = plot.figax()

ax.plot(xx, hc)

gwb = spectra['gwb'][idx]
print(gwb.shape)
gwb = gwb[:num_freqs, :]
print(gwb.shape)
med = np.median(gwb, axis=-1)
print(med)

cc, = ax.plot(xx, med)
cc = cc.get_color()
ax.fill_between(xx, *np.percentile(gwb, [25, 75], axis=-1), alpha=0.2, color=cc)

plt.show()

# OLDER METHODS

## Compute the mean and std from all spectra realizations
    At each point in parameter space, we need to find the mean value and the standard deviation from all of the realizations that we have.

In [ ]:
## NOTE - Only need to train GP on number of frequencies in PTA analysis !
gwb_spectra = spectra['gwb'][:,:30,:]**2

# Find all of the zeros and set them to be h_c = 1e-20
low_ind = np.where(gwb_spectra < 1e-40)
gwb_spectra[low_ind] = 1e-40


# Find mean over 100 realizations
mean = np.log10(np.mean(gwb_spectra, axis=-1))

# Smooth Mean Spectra
## NOTE FOR LUKE - HOW MUCH SMOOTHING DO WE WANT TO DO ?
smooth_mean = ssig.savgol_filter(mean, 7, 3)

# Find std
err = np.std(np.log10(gwb_spectra), axis=-1)

if np.any(np.isnan(err)):
    print('Got a NAN issue')

In [ ]:
## Here is an example plot of the smoothed mean, the mean and standard deviation
## and all of the spectra realizations, for a random point in parameter space.

# Choose a gwb
ind = 0

for ii in range(spectra['gwb'].shape[-1]):
    plt.loglog(spectra['fobs'][:30]*YR, spectra['gwb'][ind,:30,ii]**2, color='C0', alpha=0.3, zorder=0)
plt.loglog(spectra['fobs'][:30]*YR, spectra['gwb'][ind,:30,0]**2, color='C0', alpha=0.3, zorder=0, label='50 Spectra')
plt.loglog(spectra['fobs'][:30]*YR, 10**mean[ind], color='C1', label='Mean')
plt.loglog(spectra['fobs'][:30]*YR, 10**smooth_mean[ind], color='C3', label='Smoothed Mean')
plt.fill_between(spectra['fobs'][:30]*YR, 10**(mean[ind]-err[ind]), 10**(mean[ind]+err[ind]), color='C1', alpha=0.5)
plt.legend(loc=2)
plt.xlabel(r'GW Frequency [yr$^{-1}$]')
plt.ylabel(r'$h_{c}^{2}$')

In [ ]:
print((np.array(spectra['fobs'])[:30]*YR).max())
print((np.array(spectra['fobs'])*YR).min())

## Train GP

    The next step is to set up the GP class.
    Things to note:
        - need to make sure that the GP has the same dimensionality as the parameter space from the spectra.
        - the GPs work better when they are trained on zero-mean data, so it's very important that we remove the mean values for the spectra at each frequency, BUT these values HAVE TO BE SAVED, because they are required to extract meaningful information back out of the GP once it is trained!

In [ ]:
# Define a GP class containing the kernel parameter priors and a log-likelihood

class gaussproc(object):
    
    def __init__(self, x, y, yerr=None, par_dict = None):
        
        self.x = x
        self.y = y
        self.yerr = yerr
        self.par_dict = par_dict
        
        # The number of GP parameters is one more than the number of spectra parameters.
        self.pmax = np.array([20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]) # sampling ranges
        self.pmin = np.array([-20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0])
        self.emcee_flatchain = None
        self.emcee_flatlnprob = None
        self.emcee_kernel_map = None
    
    def lnprior(self, p):
    
        logp = 0.
    
        if np.all(p <= self.pmax) and np.all(p >= self.pmin):
            logp = np.sum(np.log(1/(self.pmax-self.pmin)))
        else:
            logp = -np.inf

        return logp

    def lnlike(self, p):

        # Update the kernel and compute the lnlikelihood.
        a, tau = np.exp(p[0]), np.exp(p[1:])
        
        lnlike = 0.0
        try:
            gp = george.GP(a * kernels.ExpSquaredKernel(tau,ndim=len(tau)))
            #gp = george.GP(a * kernels.Matern32Kernel(tau))
            gp.compute(self.x , self.yerr)
            
            lnlike = gp.lnlikelihood(self.y, quiet=True)
        except np.linalg.LinAlgError:
            lnlike = -np.inf
        
        return lnlike
    
    def lnprob(self, p):
        
        return self.lnprior(p) + self.lnlike(p)

In [ ]:
## Load in the spectra data!

# The "y" data are the means and errors for the spectra at each point in parameter space
yobs = smooth_mean.copy() #mean.copy()
yerr = err.copy()
GP_freqs = spectra['fobs'][:30].copy()
GP_freqs *= YR

## Find mean in each frequency bin (remove it before analyzing with the GP) ##
# This allows the GPs to oscillate around zero, where they are better behaved.
yobs_mean = np.mean(yobs,axis=0)
# MAKE SURE TO SAVE THESE VALUES - THE GP IS USELESS WITHOUT THEM !
np.save('./Luke_Spectra_MEANS.npy', yobs_mean)

yobs -= yobs_mean[None,:]

### Note on saving the means
I think that this .npy file is not needed, the means are saved as an attribute of the `gaussproc` objects in the `gp_george` list [here](#Save-training-information)

In [ ]:
pars = spectra['parameters'].attrs['ordered_parameters']

## The "x" data are the actual parameter values
xobs = np.zeros((spectra['gwb'].shape[0], len(pars)))

# [gsmf_phi0, hard_gamma_inner, hard_gamma_outer, hard_rchar, hard_time, mmb_amp]
for ii in range((spectra['gwb'].shape[0])):
    for k, par in enumerate(pars):
        xobs[ii,k] = spectra['sample_params'][ii,k]
        
# Put mmb_amp in logspace
xobs[:, -1] = np.log10(xobs[:, -1])

In [ ]:
# Instanciate a list of GP kernels and models [one for each frequency]

gp_george = []
k = []

# Create the parameter dictionary for the gp objects
par_dict = dict()
for ind, par in enumerate(pars):
    par_dict[par] = {"min": np.min(xobs[:, ind]),
                     "max": np.max(xobs[:, ind])}
    
for freq_ind in range(len(GP_freqs)):
    
    gp_george.append(gaussproc(xobs,yobs[:,freq_ind],yerr[:,freq_ind], par_dict))
    k.append( 1.0 * kernels.ExpSquaredKernel([2.0,2.0,2.0,2.0,2.0,2.0],ndim=6) )
    num_kpars = len(k[freq_ind])

In [ ]:
# Sample the posterior distribution of the kernel parameters 
# to find MAP value for each frequency. 

# THIS WILL TAKE A WHILE... (~ 5 min per frequency on 18 cores)

sampler = [0.0]*len(GP_freqs)
nwalkers, ndim = 36, num_kpars
for freq_ind in range(len(GP_freqs)):
    # Parellize emcee with nwalkers //2 or the maximum number of processors available, whichever is smaller
    with Pool(min(nwalkers // 2, cpu_count()) ) as pool:
        t_start = time.time()

        # Set up the sampler.
        sampler[freq_ind] = emcee.EnsembleSampler(nwalkers, ndim, gp_george[freq_ind].lnprob, pool=pool)

        # Initialize the walkers.
        p0 = [np.log([1.,1.,1.,1.,1.,1., 1.]) + 1e-4 * np.random.randn(ndim)
              for i in range(nwalkers)]

        print(freq_ind, "Running burn-in")
        p0, lnp, _ = sampler[freq_ind].run_mcmc(p0, int(750))
        sampler[freq_ind].reset()

        print(freq_ind, "Running second burn-in")
        p = p0[np.argmax(lnp)]
        p0 = [p + 1e-8 * np.random.randn(ndim) for i in range(nwalkers)]
        p0, _, _ = sampler[freq_ind].run_mcmc(p0, int(750))
        sampler[freq_ind].reset()

        print(freq_ind, "Running production")
        p0, _, _ = sampler[freq_ind].run_mcmc(p0, int(1500))

        print('Completed in {} min'.format((time.time()-t_start)/60.) , '\n')


In [ ]:
## Let's take a look at the posterior distribution of the 
# kernel parameters at a frequency [ind] of our choice.

ind = 0

fig = corner.corner(sampler[ind].flatchain,bins=50)
plt.show()

## Save training information

In [ ]:
## Populate the GP class with the details of the kernel 
## MAP values for each frequency.

for ii in range(len(GP_freqs)):
    
    gp_george[ii].chain = None 
    gp_george[ii].lnprob = None 
    
    gp_george[ii].kernel_map = sampler[ii].flatchain[np.argmax(sampler[ii].flatlnprobability)] 
    #print(ii, gp_george[ii].kernel_map)
    
    # add-in mean yobs (freq) values
    gp_george[ii].mean_spectra = yobs_mean[ii]

In [ ]:
## Save the trained GP as a pickle to be used with PTA data!
gp_file = "trained_gp_" + spectra_file.stem + ".pkl"
with open(gp_file, "wb") as gpf:
    pickle.dump(gp_george, gpf)

## Testing the GP
    The following is some example code looking at how to extract predictions from the GP and test it against the input data.

In [ ]:
# Start with Spectra from library
# Modify this as necessary for your library
spectra_file = Path('./spec_libraries/hard04b_n1000_g100_s40_r50_f40/sam-lib_hard04b_2023-01-23_01_n1000_g100_s40_r50_f40.hdf5')
spectra = h5py.File(spectra_file, 'r')

gwb_spectra = spectra['gwb'][:,:30,:]**2

# Find all of the zeros and set them to be h_c = 1e-20
low_ind = np.where(gwb_spectra < 1e-40)
gwb_spectra[low_ind] = 1e-40


# Find mean over 100 realizations
mean = np.log10(np.mean(gwb_spectra, axis=-1))

# Smooth Mean Spectra
## NOTE FOR LUKE - HOW MUCH SMOOTHING DO WE WANT TO DO ?
smooth_mean = ssig.savgol_filter(mean, 7, 3)

# Find std
err = np.std(np.log10(gwb_spectra), axis=-1)

if np.any(np.isnan(err)):
    print('Got a NAN issue')

In [ ]:
# Define a GP class containing the kernel parameter priors and a log-likelihood

class gaussproc(object):
    
    def __init__(self, x, y, yerr=None, par_dict = None):
        
        self.x = x
        self.y = y
        self.yerr = yerr
        self.par_dict = par_dict
        
        # The number of GP parameters is one more than the number of spectra parameters.
        self.pmax = np.array([20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]) # sampling ranges
        self.pmin = np.array([-20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0])
        self.emcee_flatchain = None
        self.emcee_flatlnprob = None
        self.emcee_kernel_map = None
    
    def lnprior(self, p):
    
        logp = 0.
    
        if np.all(p <= self.pmax) and np.all(p >= self.pmin):
            logp = np.sum(np.log(1/(self.pmax-self.pmin)))
        else:
            logp = -np.inf

        return logp

    def lnlike(self, p):

        # Update the kernel and compute the lnlikelihood.
        a, tau = np.exp(p[0]), np.exp(p[1:])
        
        lnlike = 0.0
        try:
            gp = george.GP(a * kernels.ExpSquaredKernel(tau,ndim=len(tau)))
            #gp = george.GP(a * kernels.Matern32Kernel(tau))
            gp.compute(self.x , self.yerr)
            
            lnlike = gp.lnlikelihood(self.y, quiet=True)
        except np.linalg.LinAlgError:
            lnlike = -np.inf
        
        return lnlike
    
    def lnprob(self, p):
        
        return self.lnprior(p) + self.lnlike(p)

In [ ]:
gp_file = "trained_gp_" + spectra_file.stem + ".pkl"
with open( gp_file, "rb") as f:
    gp_george = pickle.load(f)

In [ ]:
## Set-up GP predictions ##

gp = []
GP_freqs = spectra['fobs'][:30].copy()

for ii in range(len(GP_freqs)):
    gp_kparams = np.exp(gp_george[ii].kernel_map)

    gp.append(george.GP(gp_kparams[0] * \
            george.kernels.ExpSquaredKernel(gp_kparams[1:],ndim=len(gp_kparams[1:])) ) )

    gp[ii].compute(gp_george[ii].x, gp_george[ii].yerr)

In [ ]:
# Get parameter pairs
pars = list(gp_george[0].par_dict.keys()) # dictionaries since 3.6 keep their order! Nice

## The "x" data are the actual parameter values
xobs = np.zeros((spectra['gwb'].shape[0], len(pars)))

# [gsmf_phi0, hard_gamma_inner, hard_gamma_outer, hard_rchar, hard_time, mmb_amp]
for ii in range((spectra['gwb'].shape[0])):
    for k, par in enumerate(pars):
        xobs[ii,k] = spectra['sample_params'][ii,k]

## Make a realization from the GP

A reminder of the spectra parameters:
|Parameter|(min, max)|
| --- | --- |
|gsmf_phi0 |(-3.00e+00, -2.00e+00)|
|hard_gamma_inner | (-1.50e+00, -5.00e-01)
|hard_gamma_outer | (2.00e+00, 3.00e+00)
|hard_rchar | (1.00e+00, 3.00e+00)
|hard_time | (-1.00e+00, 1.00e+00)
|mmb_amp | (1.00e+08, 1.00e+09)

In [ ]:
# Choose the background to look at out of the 1,000 available
ind = 0

In [ ]:
env_param = xobs[ind,:].copy()
# Convert to log10(mmb_amp)
env_param[-1]= np.log10(env_param[-1])

rho_pred = np.zeros((len(GP_freqs),2))
for ii,freq in enumerate(GP_freqs):
    mu_pred, cov_pred = gp[ii].predict(gp_george[ii].y, [env_param])
    if np.diag(cov_pred) < 0.0:
        rho_pred[ii,0], rho_pred[ii,1] = mu_pred, 1e-5 * mu_pred
    else:
        rho_pred[ii,0], rho_pred[ii,1] = mu_pred, np.sqrt(np.diag(cov_pred))

## transforming from zero-mean unit-variance variable to rho
rho = np.array([gp_george[ii].mean_spectra for ii in range(len(GP_freqs))]) + rho_pred[:,0]

hc = np.sqrt(10**rho)

In [ ]:
## Making a plot ##

# the raw spectra #
for ii in range(spectra['gwb'].shape[-1]):
    plt.loglog(spectra['fobs'][:30], spectra['gwb'][ind,:30,ii], color='C0', alpha=0.2, zorder=0)
plt.loglog(spectra['fobs'][:30], spectra['gwb'][ind,:30,ii], color='C0', alpha=0.2, zorder=0, label='Original Spectra')

# the smoothed mean #
plt.loglog(spectra['fobs'][:30], np.sqrt(10**smooth_mean[ind]), color='C1', label='Smoothed Mean', lw=2)

# the GP realization #
plt.semilogx(GP_freqs, hc, color='C3', lw=2.5, label='GP')
plt.fill_between(GP_freqs, np.sqrt(10**(rho+rho_pred[:,1])), np.sqrt(10**(rho-rho_pred[:,1])), color='C3', alpha=0.5)


plt.xlabel('Observed GW Frequency [Hz]')
#plt.xlim(1e-9,7e-8)
plt.ylabel(r'$h_{c} (f)$')
#plt.ylim(1e-16, 1e-13)

plt.legend(loc=3)
#plt.savefig('./TrainedGP.pdf', bbox_inches='tight', dpi=500)

# Print the parameter values for this gwb
for i, par in enumerate(xobs[ind,:]):
    print(f"{pars[i]} = {xobs[ind,i]:.2E}")